In [34]:
import requests as rq
from lxml import etree
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool

In [ ]:
main_url = "http://tousu.315che.com/tousulist/serial/93/"
data = rq.get(main_url)
data.encoding = 'utf-8'
dom = etree.HTML(data.text, etree.HTMLParser(encoding='utf-8'))

In [ ]:
type_name = dom.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div[3]/ul/li[1]/div[2]/div[2]/div/a/text()")
type_url = dom.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div[3]/ul/li[1]/div[2]/div[2]/div/a/@href")
type_url[120] = "http://tousu.315che.com/tousulist/serial/93/"

In [ ]:
#爬取错误的网址(获取各个报告)
wrong_page_url = []

#获取所有投诉界面网址时出错的网址
wrong_cases_url = []

#获取详细案件信息时出错的网址
cases_wrong_url = []

#各个品牌的页面的网址
detail_url = []

#各个投诉界面的网址
cases_url = []

#最终的数据
cases_data = []

#诡异？
strange_url = []

count = 0

In [ ]:
#functions

def spider(urls,func,thread_num):
    pool = ThreadPool(thread_num)
    pool.map(func, urls)
    pool.close()
    pool.join()


def get_detail_url(url):
    try:
        detail = rq.get(url)

        if url in wrong_page_url:
            wrong_page_url.remove(url)

        detail.encoding = 'utf-8'
        dom_detail = etree.HTML(detail.text, etree.HTMLParser(encoding='utf-8'))

        num = int(dom_detail.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/span/em/text()")[0])
        if num != 0:
            #print("{} :num{}".format(url,num))
            pages = int(dom_detail.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[2]/div[3]/div/span[1]/text()")[0][1:-1])
            for page in range(1,pages+1):
                detail_url.append(url + "0/0/0/{}.htm".format(page))

    except Exception as e:
        if url not in wrong_page_url:
            wrong_page_url.append(url)
            #print(e)

def get_cases_url(url):

    try:
        detail = rq.get(url)

        if url in wrong_cases_url:
            wrong_cases_url.remove(url)

        detail.encoding = 'utf-8'
        dom_detail = etree.HTML(detail.text, etree.HTMLParser(encoding='utf-8'))

        for i in dom_detail.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[2]/div[2]/ul/li/a/@href"):
            cases_url.append(i)


    except Exception as e:

        if url not in wrong_cases_url:
            wrong_cases_url.append(url)
            #print(e)

def cases(url):

    global count

    count+=1

    result = 0

    try:

        da = rq.get(url)

        #判断网页是否正常打开
        if da.status_code == 200:

            if url in cases_wrong_url:
                    cases_wrong_url.remove(url)


            
            if len(da.text) != 26147:

                da.encoding = 'utf-8'
                dom = etree.HTML(da.text, etree.HTMLParser(encoding='utf-8'))
                type_name = dom.xpath("/html/body/div[1]/div[2]/div[1]/a[3]/text()")[0]
                name = dom.xpath("/html/body/div[1]/div[2]/div[1]/a[4]/text()")[0]
                detail = dom.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[1]/p/text()")[0]
                num = dom.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[3]/p[2]/text()")[0]
                ques = dom.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[3]/p[3]/text()")[0]
                time = dom.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[3]/p[4]/text()")[0]
                seller = dom.xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[3]/p[5]/text()")[0]
                
                #将数据存入字典
                case = {'type_name':type_name,'name':name,'detail':detail,'num':num,'ques':ques,'time':time,'seller':seller}

                #将字典存入列表
                cases_data.append(case)

                #print("This shit work ! {}".format(url))

                result = "good!"

            else:
                result = "this website is a piece of shit! It's url is {}".format(url)
        
        else:
             cases_wrong_url.append(url)
             result = "might be a good website need observation."

    except Exception as e:

        if url not in cases_wrong_url:
            cases_wrong_url.append(url)
            print(url)
            print(e)
            result = "might be a good website need observation."
    
    print("This is the {}th case and the result is {}".format(count,result))


In [ ]:
#获取某个品牌的所有页面的网址->detail_url
spider(type_url,get_detail_url,100000)
while (len(wrong_page_url) > 1):
    print("left:{}".format(len(wrong_page_url)))
    spider(wrong_page_url,get_detail_url,100000)

In [ ]:
#获取某个品牌的所有投诉界面的网址->cases_url
spider(detail_url,get_cases_url,100000)
while (len(wrong_cases_url) > 1):
    print("left:{}".format(len(wrong_cases_url)))
    spider(wrong_cases_url,get_cases_url,100000)

In [38]:
#获取各个投诉界面的信息(有问题，未解决)
spider(cases_url,cases,1000)
while (len(cases_wrong_url) > 1):
    print("left:{}".format(len(cases_wrong_url)))
    spider(cases_wrong_url,cases,1000)

left:100657
left:100657
